In [49]:
from datasets import load_dataset, Value, concatenate_datasets, ClassLabel
import random
import pickle

d = load_dataset("MU-NLPC/Calc-ape210k", "default", split="test")

In [52]:
d.features["result_float"].dtype.startswith("float")

True

In [47]:
out_col = "result_float"

# float to string rounding 4 decimal places
def batch_format(batch):
    batch[out_col] = [f"{x:.4f}" for x in batch[out_col]]
    return batch
d = d.map(
               batch_format, batched=True
            )

Map: 100%|██████████| 1785/1785 [00:00<00:00, 57964.67 examples/s]


In [48]:
d[0]

{'id': 'ape210k__00971711',
 'question': "Wang Yan's family bought a washing machine and a refrigerator, and they spent a total of 6,000 yuan. The price of the refrigerator is 3/5 of that of the washing machine. Ask for the price of the washing machine.",
 'question_chinese': '王艳家买了一台洗衣机和一台电冰箱，一共花了6000元，电冰箱的价钱是洗衣机的(3/5)，求洗衣机的价钱．',
 'chain': '<gadget id="calculator">3 / 5</gadget>\n<output>3/5 = around 0.6</output>\n\n<gadget id="calculator">1 + (3/5)</gadget>\n<output>8/5 = around 1.6</output>\n\n<gadget id="calculator">6_000 / (8/5)</gadget>\n<output>3_750</output>\n\n<result>3_750</result>',
 'result': '3_750',
 'result_float': 3750.0,
 'equation': 'x=6000/(1+(3/5))'}

In [44]:
out_col = "label"
int2str = d.features[out_col].int2str
# change to int dtype
d = d.cast_column(out_col, Value('int32'))
def map_function(batch):
    batch[out_col] = [int2str(i) for i in batch[out_col]]
    return batch
d = d.map(map_function, batched=True)

d[0]

Map: 100%|██████████| 400/400 [00:00<00:00, 22205.01 examples/s]


{'premise': 'My body cast a shadow over the grass.',
 'choice1': 'The sun was rising.',
 'choice2': 'The grass was cut.',
 'question': 'cause',
 'idx': 0,
 'label': 'choice1'}

In [3]:
d.push_to_hub("jeggers/words_length_short")


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.50s/it]
c:\Users\jorin\.conda\envs\reft\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jorin\.cache\huggingface\hub\datasets--jeggers--words_length_short. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/jeggers/words_length_short/commit/e5d2d0acdd02479eb87029b8096688507c0b705b', commit_message='Upload dataset', commit_description='', oid='e5d2d0acdd02479eb87029b8096688507c0b705b', pr_url=None, pr_revision=None, pr_num=None)